In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install pyforest
import pyforest
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [ ]:
df = pd.read_csv("../input/parkinson-disease-detection/Parkinsson disease.csv")
df

In [ ]:
df.groupby('status').count()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
def plot_data(df,feature):
    plt.figure(figsize=(10,6))
    plt.subplot(1,2,1)
    df[feature].hist()
    plt.subplot(1,2,2)
    stat.probplot(df[feature],dist='norm',plot=pylab)
    plt.show()

In [ ]:
features = df.columns.tolist()
features.remove('name')

In [ ]:
import scipy.stats as stat
import pylab

In [ ]:
corr = df.corr()
plt.subplots(figsize=(30,30))

sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
         annot = True,
           fmt='.1g')

In [ ]:
drop_cols = ['MDVP:Jitter(Abs)','MDVP:RAP','MDVP:PPQ','Jitter:DDP','NHR','MDVP:Shimmer(dB)','Shimmer:APQ3','Shimmer:APQ5','MDVP:APQ','Shimmer:DDA','spread1']
df.drop(columns = drop_cols,axis=1, inplace=True)
df

In [ ]:
corr = df.corr()
plt.subplots(figsize=(30,30))

sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
         annot = True,
           fmt='.1g')

In [ ]:
y = df['status']
y

In [ ]:
cols = ['name','status']
X = df.drop(columns = cols,axis = 1)
X

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, train_size=0.75, random_state=42,stratify=y)
y_test.value_counts()

In [ ]:
y_train.value_counts()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_params = {
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [5,10,50],
            'max_depth': [2,5,7,10],
            'min_samples_split': [50,70,100]
        }
    },
    'Decision_Tree' :{
        'model': DecisionTreeClassifier(),
        'params':{
            'max_depth': [2,5,7,10],
            'min_samples_split': [50,70,100]
        }
    }
    
    
}

In [ ]:
pd.set_option("display.max_colwidth", -1)
from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=6, return_train_score=False,scoring='f1_macro')
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df3 = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df3

In [ ]:
def predict(ml_model):
    import sklearn.metrics as metrics
    model=ml_model.fit(X_train,y_train)
    print('Training score : {}'.format(model.score(X_train,y_train)))
    print('Testing score : {}'.format(model.score(X_test,y_test)))
    y_prediction=model.predict(X_test)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test,model.predict(X_test))
    plt.figure(figsize = (15,10))
    sns.heatmap(cm, annot=True)
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    print(metrics.classification_report(y_test, y_prediction))

In [ ]:
model = [RandomForestClassifier(),DecisionTreeClassifier(),RandomForestClassifier(max_depth=10,min_samples_split=50,n_estimators=50),DecisionTreeClassifier(max_depth=2,min_samples_split=50)]
for i in model:
    print(i)
    predict(i)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
import pickle
with open('Pearkinson_Prediction_model.pickle','wb') as f:
    pickle.dump(model,f)